In [11]:
from configparser import ConfigParser, ExtendedInterpolation
from functools import reduce
from itertools import product

import h5py
import pandas as pd

from spikelib.utils import check_directory, datasets_to_array

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

idx =  pd.IndexSlice

In [7]:
exp_name = config['EXP']['name']
processed_file = config['FILES']['processed']
outút_folder = config['REPORT']['spreadsheet']

print('PROCESSED FILE: ' + processed_file)

PROCESSED FILE: /home/vnc/Downloads/MR-0061/data/processed_protocols/MR-0061__analysis_of_protocols.hdf5


In [15]:
df_list = []
with h5py.File(processed_file)  as panalysis:
    flash_group = '/flash/'
    for nd in panalysis[flash_group]:
        for intensity in panalysis[flash_group + nd]:
            base_group = '{}{}/{}/'.format(flash_group, nd, intensity)
            flash_charg = base_group + 'char/'
            flash_typeg = base_group + 'type/'
            col_name = [k.strip() for k in panalysis[flash_charg].attrs['col_name'].split(',')]
            char_array, keys = datasets_to_array(panalysis[flash_charg])
            type_array, type_keys = datasets_to_array(panalysis[flash_typeg])
            columns = pd.MultiIndex.from_product([[nd], [intensity], col_name], names=['nd', 'intensity', 'char'])
            df = pd.DataFrame(data=char_array, index=keys, columns=columns)
            df[(nd, intensity, 'type')] = pd.Series(type_array, index=type_keys)
            df_list.append(df)
char_data = reduce(lambda x, y: x.join(y, how='outer'), df_list)

In [25]:
char_data.to_csv('{}{}_flash_char.csv'.format(outút_folder, exp_name))